In [ ]:
import json
from city_code import CITY_CODE, REGION
import requests

HEADERS={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}



def get_brief_data(county) -> dict:
    
    # get all pages count, and loop them to fetch all data in a county
    def get_all_pages_count(county):
        # each page has 20 data
        formatted_url = f'https://newhouse.591.com.tw/home/housing/search?rid={county}&sid=&sort=1&bid=3216'
        response = requests.get(
            url=formatted_url,
            headers=HEADERS
        )
        dict_json = json.loads(response.text)

        total_count = dict_json['data']['total']
        pages_count = total_count // dict_json['data']['per_page'] + 1

        return pages_count

    
    pages = get_all_pages_count(county)
    print(f"There are {pages} pages")
                          
    result = []
    for page in range(pages):
        formatted_url = f'https://newhouse.591.com.tw/home/housing/search?rid={county}&sid=&sort=1&page={page+1}&bid=3216'
        response = requests.get(
            url=formatted_url,
            headers=HEADERS
        )
        dict_json = json.loads(response.text)
        brief_data = dict_json['data']['items']
        # result.update(brief_data)
        result += brief_data
        # print(f"Result: {result}")
    print(len(result))

    return result
    
def lambda_handler(event, context):
    # TODO implement
    
    region = event["region"]
    counties = REGION[region]
    # county = counties[0]
    
    for county in counties:
        print(f"county={county}")
        
        brief_data = get_brief_data(county)
        with open(f'{county}-brief.json', 'w') as outfile:
            json.dump(brief_data, outfile)

    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }





# Caller when AWS executed
lambda_handler({"region": "whole"}, None)

In [2]:
# Read all jsons in Brief folder to combine a whole brief data
from pathlib import Path
import json
all_json = []

for json_file in Path('./brief-jsons/').glob('*.json'):
    # print(json_file)
    with open(json_file) as jf:
        # all_json.update(json.load(jf))
        # print(json.load(jf))
        all_json += json.load(jf)

In [25]:
# print(len(all_json))

def find_all_columns(json_data):
    columns = set()

    for house in json_data:
        for column in house:
            columns.add(column)

    return columns

smaller_json = all_json
# print(smaller_json)
columns = find_all_columns(smaller_json)

In [26]:
import pandas as pd
def create_df(columns, json_data):
    data_to_df = {}

    for column in columns:
        data_to_df[column] = []
    # fill in each data
    for data in json_data:
        # for column in data:
        for column in columns:
            # key is column
            # value is data[column] if exists
            try:
                data_to_df[column].append(str(data[column]))
            except KeyError:
                data_to_df[column].append("")
    data_to_df = pd.DataFrame.from_dict(data_to_df)
    return data_to_df

    
    # return df

df = create_df(columns, smaller_json)

In [27]:
df.to_csv('20231004-brief-all-data.csv',encoding='utf-8')
df

,browsenum_week,roster_preferential_ad,browsenum_month,is_roster,regionid,im_reply_avg_time,is_package,updatetime,benefit_str,purpose_str,...,native_orderno,community_id,plus_orderno,cover,area,event_click_url,isvip,is_full,phone,section
0,,,,,1,,,2023-09-28,,工業用,...,,6766,,https://img1.591.com.tw/house/2022/01/17/16424...,32~60坪,,,0,0986-851-088,北投區
1,,,,,1,,,2023-09-08,,工商用,...,,5868908,,https://img1.591.com.tw/house/2022/01/24/16430...,168~1000坪,,,0,0986-851-088,士林區
2,,,,,1,,,2023-08-18,,住家用,...,,104495,,https://img1.591.com.tw/house/2022/11/10/16680...,77~99坪,,,1,0986-851-088,北投區
3,,,,,1,,,2023-09-22,,住家用,...,,49585,,https://img1.591.com.tw/house/2021/10/05/16334...,63~70坪,,,1,0986-851-088,北投區
4,,,,,1,,,2023-08-28,,住家用,...,,101464,,https://img1.591.com.tw/house/2019/02/20/15506...,42~44坪,,,0,0986-851-088,文山區
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4431,,,,,8,,,2023-08-21,,住家用,...,,5923260,,https://img2.591.com.tw/house/2023/03/30/16801...,,,,0,,清水區
4432,,,,,8,,,2023-08-17,,住家用,...,,5907886,,https://img1.591.com.tw/house/2023/03/31/16802...,,,,0,,沙鹿區
4433,,,,,8,,,2023-07-31,,住家用,...,,5894587,,https://img1.591.com.tw/house/2023/03/30/16801...,,,,0,,梧棲區
4434,,,,,8,,,2023-08-02,,住家用,...,,5889015,,https://img1.591.com.tw/house/2023/03/30/16801...,,,,0,,清水區


In [42]:
# Create detail data json
import requests

# df[["hid", "build_name"]]  # 113969, 130095, 125351

HEADERS={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36',
    'device': 'pc'
}

def get_detail_data(hid):
    url = f"https://bff.591.com.tw/v1/housing/detail-info?id={hid}&is_auth=0"

    response = requests.get(
            url=url,
            headers=HEADERS
        )
    dict_json = json.loads(response.text)
    return  dict_json['data']


# get_detail_data(113969)
all_detail_data = []
for i in df.hid:
    if i:
        all_detail_data.append(get_detail_data(i))
    else:
        all_detail_data.append({})
     
print(len(all_detail_data))  # Should be 4436


    

4436


In [45]:
# Create detail data with list

columns = find_all_columns(all_detail_data)
print(columns)
df = create_df(columns, all_detail_data)

{'project_pay', 'build_type_name', 'shop_price', 'remark', 'build_area', 'regionid', 'jbrate', 'ratio', 'lend_rate', 'office_price', 'down_pay', 'license', 'price_unit', 'park_price', 'address', 'section', 'purpose_other_name', 'reception_address', 'other_build', 'floor', 'region', 'is_upscale', 'room_rule', 'purpose', 'park_planning', 'addr_number', 'terrace', 'use_license', 'sell_time', 'surrounding', 'structural_engine', 'sectionid', 'price_total', 'special_purpose', 'layout', 'build_type', 'manage_cost', 'households', 'manage_committee', 'price', 'map', 'meta', 'transportation', 'share_num', 'hid', 'direction_rule', 'park_style', 'purpose_name', 'facility', 'width_deep', 'build_intro', 'property_company', 'building_design', 'park_ratio', 'reception_map', 'fav_num', 'land_division', 'label', 'build_name', 'decorate', 'base_area', 'deal_time'}


In [47]:
df.to_csv('20231006-detail-all-data.csv',encoding='utf-8')